## Finetuning ResNet18 for classification

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, random_split, Subset
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn.model_selection import train_test_split

import time
import os
import requests
import random
from io import BytesIO
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tempfile import TemporaryDirectory

cudnn.benchmark = True
plt.ion()   # interactive mode

In [2]:
print("pytorch", torch.__version__)
print("cuda:", torch.version.cuda)

pytorch 2.5.1
cuda: 11.8


In [27]:
import gc
torch.cuda.empty_cache()
gc.collect()

0

In [25]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)

cuda:0


In [5]:
%cd D:\\Images\\Training_images

D:\Images\Training_images


c:\Users\sirifris\.conda\envs\poli_reco\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [6]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)), # resize all images 
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1), 
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.2),
    transforms.RandomPerspective(distortion_scale=0.2, p=0.5),
    transforms.RandomGrayscale(p=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    # Normalizing using the numbers recommended when working with ResNet18
])

In [7]:
inference_transform = transforms.Compose([
    transforms.Resize((224, 224)), # resize all images 
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    # Normalizing using the numbers recommended when working with ResNet18
])

In [8]:
data_dir = 'data'

subfolders = [f.name for f in os.scandir(data_dir) if f.is_dir()]

# Iterate over each subfolder (each species)
for subfolder in subfolders:
    subfolder_path = os.path.join(data_dir, subfolder)

    # Get a list of all image files in the subfolder
    images = [f for f in os.listdir(subfolder_path) if f.endswith(('.png', '.jpg', '.jpeg'))]

    # Select a random image from the subfolder
    random_image = random.choice(images)

    # Full path to the selected image
    image_path = os.path.join(subfolder_path, random_image)

    # Open and display the image<
    image = Image.open(image_path)
    # print(f"Displaying one random image from {subfolder}: {random_image}")
    # display(image)

In [9]:
train_dataset = datasets.ImageFolder(data_dir, transform=train_transform)
val_test_dataset = datasets.ImageFolder(data_dir, transform=inference_transform)

In [10]:
# Extract indices and corresponding labels
indices = np.arange(len(train_dataset))
labels = np.array([label for _, label in train_dataset.imgs])

# Split into training (70%) and combined validation/test (30%)
train_indices, val_test_indices, train_labels, val_test_labels = train_test_split(
    indices, labels, test_size=0.3, stratify=labels, random_state=42)

# Split the combined validation/test set into validation (15%) and test (15%)
val_indices, test_indices, _, _ = train_test_split(
    val_test_indices, val_test_labels, test_size=0.5, stratify=val_test_labels, random_state=42)

# Create samplers for each set
train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)
test_sampler = SubsetRandomSampler(test_indices)

# Create DataLoader for each set
train_loader = DataLoader(train_dataset, batch_size=32, sampler=train_sampler, num_workers=2)
val_loader = DataLoader(val_test_dataset, batch_size=32, sampler=val_sampler, num_workers=2)
test_loader = DataLoader(val_test_dataset, batch_size=32, sampler=test_sampler, num_workers=2)

In [11]:
# Load a pre-trained ResNet18 model
model = models.resnet18(pretrained=True)

# Modify the fully connected layer for 17 classes
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 22)

# Move the model to the device
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Using a scheduler to adjust the learning rate dynamically
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

c:\Users\sirifris\.conda\envs\poli_reco\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\sirifris\.conda\envs\poli_reco\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [12]:
# Training loop
num_epochs = 10 # Doing 20 epochs
best_val_acc = 0.0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss = running_loss / len(train_loader.sampler)
    train_acc = correct / total

    scheduler.step()

    # Validation
    model.eval()
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_acc = val_correct / val_total

    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}')

    # Save the best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), 'best_model.pth')

Epoch 1/10, Train Loss: 1.4010, Train Acc: 0.5657, Val Acc: 0.6531
Epoch 2/10, Train Loss: 0.6519, Train Acc: 0.7887, Val Acc: 0.7900
Epoch 3/10, Train Loss: 0.4325, Train Acc: 0.8665, Val Acc: 0.8939
Epoch 4/10, Train Loss: 0.3325, Train Acc: 0.8941, Val Acc: 0.9185
Epoch 5/10, Train Loss: 0.2502, Train Acc: 0.9230, Val Acc: 0.9472
Epoch 6/10, Train Loss: 0.1165, Train Acc: 0.9673, Val Acc: 0.9953
Epoch 7/10, Train Loss: 0.0615, Train Acc: 0.9830, Val Acc: 0.9953
Epoch 8/10, Train Loss: 0.0549, Train Acc: 0.9837, Val Acc: 0.9974
Epoch 9/10, Train Loss: 0.0417, Train Acc: 0.9878, Val Acc: 0.9969
Epoch 10/10, Train Loss: 0.0331, Train Acc: 0.9909, Val Acc: 0.9984


In [13]:
# Loading the best model
model.load_state_dict(torch.load('best_model.pth'))

# Evaluate on the test data
model.eval()
test_correct = 0
test_total = 0
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_acc = test_correct / test_total
print(f'Test Accuracy: {test_acc:.4f}')

C:\Users\sirifris\AppData\Local\Temp\ipykernel_33312\1818604445.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))


Test Accuracy: 0.9974


In [14]:
from sklearn.metrics import confusion_matrix, classification_report

# Cassification report:
print(classification_report(all_labels, all_preds, target_names=val_test_dataset.classes))

                        precision    recall  f1-score   support

         Anthony Fauci       1.00      1.00      1.00        87
             Bent Høie       1.00      1.00      1.00        87
            Bill Gates       1.00      1.00      1.00        87
   Camilla Stoltenberg       1.00      1.00      1.00        87
          Donald Trump       1.00      0.99      0.99        87
          Erna Solberg       1.00      1.00      1.00        87
 Espen Rostrup Nakstad       1.00      0.97      0.98        87
       Fredrik Solvang       1.00      1.00      1.00        87
             Joe Biden       1.00      1.00      1.00        87
      Jonas Gahr Støre       1.00      1.00      1.00        87
        Lan Marie Berg       1.00      1.00      1.00        87
 Random People Walking       1.00      1.00      1.00        87
        Sylvi Lysthaug       1.00      1.00      1.00        87
Trygve Slagsvold Vedum       1.00      1.00      1.00        87
     cuasual snapshots       0.99      

In [15]:
import seaborn as sns

cm = confusion_matrix(all_labels, all_preds)

plt.figure(figsize=(10,8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show

ModuleNotFoundError: No module named 'seaborn'

# Classify the actual data

In [17]:
model = models.resnet18(pretrained = True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 22)

model.load_state_dict(torch.load('best_model.pth', map_location=device))
model.eval()

C:\Users\sirifris\AppData\Local\Temp\ipykernel_33312\172419626.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth', map_loc

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [18]:
image_folder = 'D:\\Images\\Covid\\output_images_detected'
output_folder = 'D:\\Images\\Covid\\classified_images'

In [20]:
image_paths = [os.path.join(image_folder, img) for img in os.listdir(image_folder) if img.endswith(('.png', '.jpg', '.jpeg'))]

In [21]:
class_names = [
    'Jonas Gahr Støre',
    'Erna Solberg', 
    'Bent Høie',
    'Lan Marie Berg',
    'Sylvi Lysthaug',
    'Espen Rostrup Nakstad',
    'Trygve Slagsvold Vedum',
    'Camilla Stoltenberg',
    'Fredrik Solvang',
    'Donald Trump',
    'Joe Biden',
    'Anthony Fauci',
    'Bill Gates',
    'Random People Walking',
    'random people',
    'random facebook post',
    'random twitter post',
    'casual snapshots',
    'lifestyle photos',
    'everyday moments',
    'food photo',
    'nature photo'
]

In [22]:
from pathlib import Path

for class_name in class_names:
    class_folder = Path(output_folder) / class_name
    class_folder.mkdir(parents=True, exist_ok=True)

In [23]:
import shutil

for image_path in image_paths:
    try:
        image = Image.open(image_path).convert('RGB')
        input_tensor = inference_transform(image).unsqueeze(0)

        with torch.no_grad():
            outputs = model(input_tensor)
            _, predicted = torch.max(outputs, 1)

        predicted_class = class_names[predicted.item()]

        target_folder = Path(output_folder) / predicted_class

        shutil.copy(image_path, target_folder)

    except Exception as e:
        print(f"Error processing {image_path}: {e}")

print(f"All images have been classified")


KeyboardInterrupt: 